## Part 1: Autoencoders

There was one slide from Mike's lecture about how weights evolve
Encodes and decodes input from same thing
Unsupervised ML task (no label) - 

What we care about is output of hidden layer = compressed data = embedding = latent space
Same thing as training any other neural net 
Non-linear dimensionality reduction (with sigmoid/ReLU activation functions) - not like PCA or others
Can ignore decoder part and take embedding and do linear reduction - because they are "better" features

The decoder is an example of a generative model - because they are dimensionally increasing

* The purpose of this autoencoder is to produce a "binding" score between 0 and 1 for prob of being TF binding site
* Note that training data is heavily weighted towards negative data
* Should downsample neg data or duplicate positive  

  
* Part 2: Adaptation of code but not necessarily
* Part 3: Cross evaluation - full credit should be from scratch - can use scikit learn but will be 1-2 points
* Part 4: Attempt is good - just do hyperparameter optimization + try something cool - explain what you tried, how it did, and why you chose that
* Part 5: Evaluate final model and see if your NN is working well (blinded on our side, TAs have labels)

  
* can create NN class, tell it dimension of output layer, other factors, etc
* for encoding DNA data, one-hot encoding and flatten (eg [1,0,0,0] for A, [0,0,0,1] for G)

* Why do you need a cost function that is readily differentiable? To get the simplest gradient possible since it is repeatedly calcualted during training
* ie. cost function is how close you are to the target, the gradient is how fast you're approaching the target

Training Goal: optimize J(theta) over choices of theta
* Initialize weights (theta = t0, t1, ... ) = vector of small random values
* Define a cost function J(theta)
* Define a gradient dtheta(J(theta))
* For t=1 ... max time --- set theta to (theta - alpha(dtheta)) --- what this is doing is moving the weights a little
* Halt on convergence (when J(theta) doesn't change much anymore) or time out and report theta

* Can perform sensitivity analysis on how changing parameters / training set changes ROC



array([[0.61924043, 0.03240046, 0.12537537, 0.05353615, 0.02873017,
        0.10444717, 0.00878742, 0.17182233],
       [0.01783419, 0.29022285, 0.0534807 , 0.11589074, 0.01506372,
        0.13276782, 0.29178005, 0.11815918],
       [0.10646459, 0.04166288, 0.18054344, 0.1565528 , 0.21384123,
        0.0502957 , 0.03777032, 0.04040759],
       [0.02720728, 0.07754488, 0.14678251, 0.20005386, 0.18649696,
        0.04898576, 0.10162535, 0.03090916],
       [0.04330614, 0.03138022, 0.25370597, 0.25858886, 0.54762191,
        0.0282026 , 0.04802834, 0.0150942 ],
       [0.11606019, 0.16307883, 0.06781575, 0.07635392, 0.01616379,
        0.15268816, 0.10203679, 0.1867526 ],
       [0.0076296 , 0.30358028, 0.06227495, 0.16434759, 0.03252095,
        0.10398725, 0.39941341, 0.07176241],
       [0.19381545, 0.15543503, 0.06170166, 0.05902934, 0.00971223,
        0.17823356, 0.0747741 , 0.24993524]])

In [375]:

for l in n.layers:
    print(l.shape)
    
print()
    
for l in n.weights:
    print(l.shape)

print()
    
for l in n.z_values:
    print(l.shape)
    
    

(8, 7)
(3, 7)
(8, 7)

(3, 8)
(8, 3)

(8, 7)
(3, 7)
(8, 7)


1. let C be the error / loss (eg. calculated from BCE)

2. let a(L) = activation of L = sigmoid(z(L)) where z(L) = w(L)a(L-1) + b(L)

3. let dC/dw(L) be how sensitive the cost function is to w(L)
- since the cost function is dependent on a(L) which is dependent on z(L), dC/dw(L) = dz(L)/dw(L) * da(L)/dz(L) * dC/da(L) by the chain rule
- where dz(L)/dw(L) = dx activation* = a(L-1)
- where da(L)/dz(L) = dx sigmoid function = d_sigmoid(Z(L))
- where dC/da(L) = dx cost function = d_BCE(a(L))

\* Since Z(L) = w(L) * a(L-1) + b(L)

4. so dC/dw(L) = a(L-1) * d_sigmoid(Z(L)) * d_BCE(a(L)) 

5. dC/da(L-1) = dz(L)
 
6. and dC/db = dz(L)/db(L) * da(L)/dz(L) * dC/da(L)
- where dz(L)/db(L) = 1
- where da(L)/dz(L) = dx sigmoid function = d_sigmoid(Z(L))
- where dC/da(L) = dx cost function = d_BCE(a(L))



In [285]:
# δL=gradient(a of C)∘sigmoid'(z(L))
# δi=W(i+1).transpose() * δ(i+1) ∘sigmoid'(z(i))

# dC/db(l,j) = δ(i,j)
# dC/dw(l,j,i) = a(i-1, k) δ(i,j)



TODO: for batch gradient descent, shuffle training examples and divide into batches for backprop




In [15]:
    
        # start with last layer and get delta L
        # x(L) are the outputs (= the x values of the last layer)
        # f'L is the derivative of the activation function for that layer
        # δL=(x(L)−t)∘f′L(W(L)x(L−1))
        
        # for all the other layers 1 ... i , get the delta for i
        # f'i is the derivative of the activaiton function in that layer (sigmoid) 
        # with respect to the current weights (between current layer and prev layer)
        # W are the weights at the position i or i+1
        # x are the values at the current layer
        # Yes that is a recursive delta(i+1) being added so make sure to keep track of all the deltas
        # δi = W(Transpose)(i+1) * δ(i+1) ∘ f′i(W(i) * x(i−1))
        
        # compute gradient of the current layer
        curr_slope = self._dxSigmoid(self.curr_layer) # <- derivative of the activation function
        
        # compute weight modifications for output layer
        delta_out = error * slope_out

        # compute weight modification for hidden layer
        # error of hidden layer is back-propagated from error at output
        delta_hidden = np.dot(delta_out,self.wo.T) * slope_hidden
        
        # NOTE: out_gradient = 2*(y-pred)*sigmoid_derivative(self.output)
        # what you're calculating is previous activaiton * loss

        #θTx = θ0+nj = 1 θjxj

        
        
        
        # update the weights with the derivative (slope) of the loss function
        # np.dot(layer, delta) = gradient
        # update the bias using the sum of the deltas
        prev_layer = self.out_layer
        for ind in range(self.hid_layers): # TODO: may be off by one, need to keep one for the input layer
            self.weights[ind] += alpha * np.dot(prev_layer, curr_delta)
            self.bias[ind] += self.delta[ind] * lr
            prev_layer = self.hid_layers[ind] # update recursively to backpropagate through the network
        
        
        
        
        
        
        
    
    def gradientDescent():
        pass


    # TODO: REMEMBER TO REPORT AVERAGE LOSS PER EPOCH
    def lossDerivative(y, pred):
        """
        Calculates the loss at each iteration using the binary cross-entropy (BCE) loss function.
        
        Loss appraoches infinity when (y - pred) -> 1, where y is the known class and pred is the 
        class predicted in the current iteration.
        
        Take derivative of 
        
        
        Parameters:
        y : int
        Actual
        pred : int
        
        """
        
        # This is calculating the gradient, which is the derivative of the loss function
        # When y = 0, returns 1 / (1-pred)
        # When y = 1, returns -1 / pred
        return (-y/pred) + ((1-y)/(1-pred)) 
    
    def fit(self, x_train, y_train, activation):
        
        # ??? Do you shuffle the data when running multiple epochs
        for epoch in self.epochs:
            # take some subset for of values and outputs
            for value, output in zip(x_train, y_train):
                self.forward(value, activation)
                #self.lossCalculation() ??? Do I need a separate function for this?
                self.backpropagate(value, output)
    
    def predict():
        pass

# https://towardsdatascience.com/nothing-but-numpy-understanding-creating-binary-classification-neural-networks-with-e746423c8d5c



IndentationError: expected an indented block (<ipython-input-15-e2717e85c2bc>, line 72)

In [42]:
def activate(weights, inputs):
    activation = weights[-1]
    print(activation)
    
    for i in range(len(weights)-1):
        activation += weights[i] * inputs[i]
    
    return activation

def dotProduct(weights, inputs):
    return np.dot(weights, inputs)
    
# Transfer neuron activation by sigmoid function
def transfer(activation):
    return 1.0 / (1.0 + exp(-activation))
    
# Forward propagate input to a network output
def forward_propagate(input_layer, network):
    
    for layer in network:
        new_inputs = []
        
        #for neuron in layer:
        
        activation = activate(layer, input_layer)
        
        return activation
    
        neuron['output'] = transfer(activation)
        new_inputs.append(neuron['output'])
            
    inputs = new_inputs
    
    return inputs

In [31]:

from math import exp
from random import seed
from random import random
 
# Initialize a network
def initialize_network(n_inputs, n_hidden, n_outputs):
    network = list()
    hidden_layer = [{'weights':[random() for i in range(n_inputs + 1)]} for i in range(n_hidden)]
    network.append(hidden_layer)
    output_layer = [{'weights':[random() for i in range(n_hidden + 1)]} for i in range(n_outputs)]
    network.append(output_layer)
    return network
 
# Calculate neuron activation for an input
def activate(weights, inputs):
    activation = weights[-1]
    for i in range(len(weights)-1):
        activation += weights[i] * inputs[i]
    return activation
 
# Transfer neuron activation
def transfer(activation):
    return 1.0 / (1.0 + exp(-activation))
 
# Forward propagate input to a network output
def forward_propagate(network, row):
    inputs = row
    
    # for each layer, update its weights
    for layer in network:
        layer = new_inputs.append(transfer(activate(layer, inputs)))

# test forward propagation
seed(1)
network = initialize_network(2, 1, 2)
for layer in network:
    print(layer)
    
row = [1, 0, None]
output = forward_propagate(network, row)
print(output)
    

[{'weights': [0.13436424411240122, 0.8474337369372327, 0.763774618976614]}]
[{'weights': [0.2550690257394217, 0.49543508709194095]}, {'weights': [0.4494910647887381, 0.651592972722763]}]
[0.6629970129852887, 0.7253160725279748]


In [13]:
import numpy as np
np.dot([1,2,3], [[1], [2], [3]])


array([14])

In [ ]:
def encodeFasta(value):
    """
    
    """
    
    pass